In [1]:
#load data file
import os
import pandas as pd
from tkinter import Tk, filedialog

# 選取多個資料檔案
root = Tk()
root.attributes('-topmost', True)  # 將主視窗設置為最上層
root.withdraw()

data_file_paths = filedialog.askopenfilenames(parent = root ,title='選擇資料檔案', filetypes=[('CSV files', '*.csv')])

all_raw_df = {}


# 循環處理每個選取的檔案
for index, data_file_dir in enumerate(data_file_paths):
    data_file_name = os.path.basename(data_file_dir)
    file_name_parts = data_file_name.split('_')
    mouseID = file_name_parts[0]
    Date = file_name_parts[1]
    schedule = file_name_parts[2]

    print("當前處理檔案:" + data_file_name)

    df = pd.read_csv(data_file_dir)
    if df.columns[0] == "Trial_number":
        print("檔案格式正確")
    else:
        print("檔案格式錯誤")

    df = df.dropna()
    all_raw_df[index] = df


當前處理檔案:G268_0817_stage3_image_150cm_12ul.csv
檔案格式正確


In [6]:
# 設定目標時長
target_duration = 60  # 單位：秒

# 重置索引為連續的整數
df.reset_index(drop=True, inplace=True)

# 將單獨或只出現兩次的 0 改為 1
for i in range(1, len(all_raw_df[0])-1):
    if all_raw_df[0]['SLAP2_acquiring'][i] == 0 and all_raw_df[0]['SLAP2_acquiring'][i-1] == 1 and all_raw_df[0]['SLAP2_acquiring'][i+1] == 1:
        all_raw_df[0].loc[i, 'SLAP2_acquiring'] = 1

# 找尋連續的 SLAP2_acquiring 為 1 的部分
consecutive_ones = (all_raw_df[0]['SLAP2_acquiring'] != all_raw_df[0]['SLAP2_acquiring'].shift()).cumsum()

# 計算每個連續部分的起始和結束時間
grouped = all_raw_df[0].groupby(consecutive_ones)
start_times = grouped['Time'].first()
end_times = grouped['Time'].last()

# 找到與目標時長相近的部分
selected_segments = []
for start, end in zip(start_times, end_times):
    duration = (end - start) / 1000
    if target_duration-1 <= duration <= target_duration + 1:
        selected_segments = selected_segments + [(start, end)]


# 創建新的 DataFrame 以存放結果
selected_data = pd.DataFrame(columns=all_raw_df[0].columns)

# 根據找到的時段將資料加入到新的 DataFrame 中
for start, end in selected_segments:
    selected_data = pd.concat([selected_data, all_raw_df[0][(all_raw_df[0]['Time'] >= start) & (all_raw_df[0]['Time'] <= end)]], axis=0)

# 重置時間為從 0 開始
selected_data['Time'] = selected_data['Time'] - selected_data['Time'].iloc[0]

# 將結果存成新的 CSV 檔案
selected_data.to_csv('selected_data.csv', index=False)
